In [1]:
from AnthropicWrapper import ClaudeChatCV
from joblib import load

In [ ]:
input_conv = load('conversation.joblib')

In [ ]:
model = "claude-3-5-sonnet-20240620"
pdf_path = "D:\Hidden Desktop\OneDrive\Cross Device\Jobs Applications\Graduating\CV.pdf"

job_role = "Machine Learning Engineer"
candidate_skill = "Entry-Level"

In [ ]:
system_prompt = f"""You are a skilled interviewer who is conducting a review on the transcript of a phone screening interview. The purpose of the screening interview is to ensure candidates who move forward to the interview stage are suitable for the role and worth interviewing.
The interviewee is a candidate for the role of {candidate_skill} {job_role}. 
The interviewee has provided a CV for your consideration. Please provide feedback on the candidate's performance and suitability for the role."""

In [ ]:
claude = ClaudeChatCV(model, system_prompt, pdf_path)

In [ ]:
print(claude.chat_with_history_doc(input()))